In [ ]:
## Import libs
import findspark
findspark.init("/home/antonis/spark-2.3.0-bin-hadoop2.7")

import os.path
import pandas
import math
import time
import scipy.stats as ss
import numpy as np
import pandas as pd

from metrics.Correlation import Correlation
from IO.Output import Output
from IO.Input import Input
from metrics.RSquare import RSquare
from DataSet.Dataset import DataSet

import pyspark
from pyspark.sql import SparkSession


import sys
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import countDistinct,avg,stddev
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                               OneHotEncoder , StringIndexer)
import DataSet.SnpsSelection as s

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import DenseVector
from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LinearSVC

from pyspark.ml.classification import NaiveBayes

from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.ml.stat import Correlation
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.linalg import DenseVector

from pyspark.ml.classification import DecisionTreeClassifier


import math
import time

from multiprocessing.dummy import Pool as ThreadPool



def writeCoef(path,snpsIds,sc,idToName,corr, name = None):
        
    if not name:
        print("give a name to file")
        return
        
    p = path + name  + " ( " + time.strftime("%d-%m-%Y") + " ).txt "  
    
    i=1
    while os.path.exists(p):
            
        p = path + name  + " ( " + time.strftime("%d-%m-%Y") + " ) " + '_' + str(i)+".txt"
        i += 1
        
    snps = []
    for i in range(len(snpsIds)):
        s = snpsIds[i]
        #snps.append(idToName[s])
        snps.append(s)
            
    print("snpsIds = ",len(snpsIds))
    print("idToName = ",len(idToName))
        
    write = open(p,'w')
    write.write("len = "+str(len(sc))+'\n')
    write.write('corr = '+str(corr)+'\n')
    for i in range(len(snps)):
            
        write.write(str(snps[i])+'\t'+str(sc[i])+'\n')
            
    write.close()


def calcCoeff(path,coefs, columns,corr,name = None):
    
    ids = {}
    ids['coef']={}
   
    print("before = ",len(set(coefs)))
    #for i in range(len(coefs)):
     #   coefs[i] = abs(coefs[i])

    print("after = ",len(set(coefs)))

    idToName = {}
    nameToId = {}

    for i in range(len(coefs)):
        nameToId[coefs[i]] = []

    for i in range(len(coefs)):
        nameToId[coefs[i]].append(i)
        idToName[i] = coefs[i]


    ids['coef']['nameToId'] = nameToId
    ids['coef']['idToName'] = idToName


    sc = sorted(coefs,reverse=True)

    top_30 = []

    for i in range(30):

        snp = ids['coef']['nameToId'][sc[i]][0]
        ids['coef']['nameToId'][sc[i]].remove(snp)
        top_30.append(columns[snp])
     #   top_30.append(snp)

    #snpReduc['low'] = top_30
    writeCoef(path,top_30,sc,columns,corr, name = name)
    
    
def showMyMetric(results,v1 = 1,v2 = 0):
    
    metrics = {}
    
    if v1 == v2:
        print("wrong values!!!v1 is the same with v2!!!!")
       
    

    tp = results.where('label == 1 and prediction ==1').count()
    fp = results.where('label == 0 and prediction ==1').count()

    tn = results.where('label == 0 and prediction ==0').count()
    fn = results.where('label == 1 and prediction ==0').count()
    
    metrics['tp'] = tp
    metrics['fp'] = fp
    metrics['tn'] = tn
    metrics['fn'] = fn
    
    return metrics
    
    
    
    
def calculateAvgMetrics(results,classLabel=1):
    
    metricss = {}
    metrics = showMyMetric(results)
    
    metricss["accuracy"] = (metrics['tp'] + metrics['tn']) / (metrics['tp'] + metrics['fp'] + metrics['tn'] + metrics['fn'])
    metricss["recal"]= metrics['tp'] / (metrics['tp'] + metrics['fn'])
    metricss["precision"] = metrics['tp'] / (metrics['tp'] + metrics['fp'])
    metricss["f1"] = 2 * ((metricss['recal'] * metricss['precision']) / (metricss['recal'] + metricss['precision']))
    
    return metricss
    
    
def split(numFold = 0, df = None):
    
    samples = {}
    labels = {}
    
    for i in range(1,numFold+1):
        labels[i] = {}
    
    if numFold == 0:
        
        print("wrong!!!!! num fold is zero (0)")
        
        return None
    
    dfCount = df.count()
    k = int (dfCount / numFold)
    
    print("df = ", dfCount)
    print("k = ",k)

    
    dfsplit = df
    
    for i in range(1, numFold):
        
        x = (k*100) / dfCount
        x = x / 100
        
        split1, split2 = dfsplit.randomSplit([x,1-x],seed=2018)
    
        labels[i]['label0'] = split1.filter('label == 0').count()
        labels[i]['label1'] = split1.filter('label == 1').count()
        
        
        dfsplit= split2
        dfCount = dfCount - k
        
        samples[i] = split1
        
        
    samples[numFold] = dfsplit
    labels[numFold]['label0'] = dfsplit.filter('label == 0').count()
    labels[numFold]['label1'] = dfsplit.filter('label == 1').count()
    
    return samples,labels


def balanedData(df,label1,label0):
    
    #label1 = df.filter("label == 1").count()
    #label0 = df.filter("label == 0").count()
    print("label0 = ", label0)
    print("label1 = ", label1)


    l0 = label1 * 2

    x = (l0 * 100) / label0


    x = x / 100

    print("label1 = ", label1)
    print("l2 = ",l0)
    print("label0 = ", label0)
    print("x = ", x)
    print()

    t0 = df.filter('label == 0')
    t1 = df.filter('label == 1')

    train0,t2 = t0.randomSplit([x,1-x],seed = 11)
    
    train_data = train0.union(t1)
    
    return train_data


def reduceDismension(train_data, test_data):
    
   
    
    return train, test


    

def crossVal(numFold = 0,samples = None, labels =None,classifier = None,name = None):
    
    avgMetrics = {}
    avgMetrics[name] = {}
    
    
    if classifier == None:
        
        print("classifier not given")
        return
    
    
    if samples == None:
        return
    
    for i in range(1,numFold + 1):
        
        print("cross = ",i)
         
        test_data = samples['test'][i]
        
      
        train = samples['train'][i]
        
        model = classifier.fit(train)
           
        results = model.transform(test_data)

        evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
        AUC = evaluate.evaluate(results)
        
        
        
        results1 = results.select('prediction','label')
    
        r = calculateAvgMetrics(results1,classLabel=1)
        r['auc'] = AUC
        avgMetrics[name][i] = r
    
    return avgMetrics


def writeResult(path,name,results=None,category='',thresh=-1, down=-1, up=-1):
    
    timee = time.strftime("%d-%m-%Y")
    
    folder = path + name  + " ( " + timee + " ) " + '_'
    #file = path + name  + '.txt'
    
    i=1
    while os.path.exists(folder):
        timee = time.strftime("%d-%m-%Y")
        folder = path + name + " ( " + timee + " ) " + '_' + str(i) + '_'
       
        i += 1
        
   # os.makedirs(folder)
    
    file = folder + name + '.txt'
    
    write = open(file,'w')
    
    write.write(timee + '\n'+ '\n')
    write.write(name + '\n')
    write.write('threshold = ' + str(thresh) + '\n')
    write.write('down = ' + str(down) +'\n')
    write.write('up = '+ str(up) + '\n')
    write.write("Category = " + category + '\n')
    for i in results.keys():
        
        #results1 = results[i]
        
        
        #for j in results:
            
        write.write(str(i) + " = " + str(results[i]) + '\n')
            
        
        
    write.close()
    
    
        

    

def writeNSemanticSnps(columns = None, snps = None, name =None):
    
    if not name:
        print("give a name to file")
        return
        
    p = path + name  + " ( " + time.strftime("%d-%m-%Y") + " ).txt"
    
    i=1
    while os.path.exists(p):
            
        p = path + name  + " ( " + time.strftime("%d-%m-%Y") + " ) " + '_' + str(i)+".txt"
        i += 1
     
    write = open(p,'w')
    
    snps = []
    for i in snps.keys():
        s = snps[i]
        write.write(columns[i] +',   ' + s + '\n')
                
            
    write.close()
    



In [ ]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
    .set("spark.driver.maxResultSize", "20g")
   
    .set('spark.graphx.pregel.checkpointInterval','-1')
    .set('spark.network.timeout','100000000')
    .set('spark.executor.heartbeatInterval','10000000'))

    


#spark = SparkSession.builder.appName('melanoma').getOrCreate()
sc = SparkContext(conf=conf)
spark = SQLContext(sc)

In [ ]:
import os

os.environ["SPARK_HOME"] = "/media/antonis/red"

In [ ]:
path = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/'

#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.001/snpCodeTest.csv'
#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.01/snp2.txt'
#pathSnp = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'
pathSnp = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/snpCodeTest1.csv'

#pathSnp = '/media/antonis/Antonis_Moulopoulos/newSet/pvalue = 0.001/snp1.txt'
#pathSnp = '/media/antonis/Antonis_Moulopoulos/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'


data = spark.read.option("maxColumns", 80000).csv(pathSnp,inferSchema=True,header=True)
data=data.withColumnRenamed('TARGET','label')




In [ ]:
print("data columns = ",len(data.columns))

features = []
for i in data.columns:
    if 'rs' not in i and i !='label':
        features.append(i)
print(len(features))
#print((features))

In [ ]:
d = data.drop('patients')
d = d.drop(*features)
print("columns = ",len(d.columns))
#print(d.columns[:-1])
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                               OneHotEncoder , StringIndexer)

assempler = VectorAssembler(inputCols=d.columns[:-1],outputCol='features')

d1 = assempler.transform(d)

d = d1.select('features','label')


In [ ]:
d.show(10)

# SAMPLES

In [ ]:
folds =5

samples, labels = split(folds,d)

train = {}
test = {}

for i in range(1,folds + 1):
        
        test[i] = samples[i]
        label0 = 0
        label1 = 0
        
        if i <= 2:
        
            train_data = samples[3].union(samples[4])
            
            label0 = labels[3]['label0']+labels[4]['label0']
            label1 = labels[3]['label1']+labels[4]['label1']
            
            s1 = 3
            s2 = 4
                   
        else:
            
            train_data = samples[1].union(samples[2])
            
            label0 = labels[1]['label0']+labels[2]['label0']
            label1 = labels[1]['label1']+labels[2]['label1']
            
            s1 = 1
            s2 = 2
            
            
        for j in range(1,folds + 1):
            
            if j != i and j!= s2 and j != s1:
                
                train_data = train_data.union(samples[j])
                label0 = label0 + labels[j]['label0']
                label1 = label1 + labels[j]['label1']
                
        train[i] = balanedData(train_data,label1,label0)
        
samples['train'] = train
samples['test'] = test

# Regressor

In [ ]:


log_reg = LogisticRegression(featuresCol='features',labelCol='label',maxIter=10)

results = crossVal(numFold = 1,samples= samples, labels = labels,classifier = log_reg)



In [ ]:
avg = {}
print
for i in results[1].keys():
    avg[i] = 0
    
for i in results.keys():
    for j in results[i].keys():
        avg[j] = avg[j] + results[i][j]
        
for i in avg.keys():
    avg[i] = float(avg[i]/folds)



writeResult('regression_pearson',results=avg,category='pearson')

In [ ]:
treeD = DecisionTreeClassifier(predictionCol='prediction', labelCol='label')

results = crossVal(numFold = folds,data = samples, labels = labels,classifier = treeD)

avg = {}

for i in results[0].keys():
    avg[i] = 0
    
for i in results.keys():
    for j in results[i].keys():
        avg[j] = avg[j] + results[i][j]
        
for i in avg.keys():
    avg[i] = float(avg[i]/10)

writeResult('allfeatures_tree',results=avg,category='all') 

In [ ]:
lsvc = LinearSVC(featuresCol='features',labelCol='label',maxIter=10)
lsvc.setThreshold(0.5)
results = crossVal(numFold = folds,data = samples, labels = labels,classifier = lsvc)

avg = {}

for i in results[0].keys():
    avg[i] = 0
    
for i in results.keys():
    for j in results[i].keys():
        avg[j] = avg[j] + results[i][j]
        
for i in avg.keys():
    avg[i] = float(avg[i]/10)

writeResult('allfeatures_svm',results=avg,category='all') 

In [ ]:
rf = RandomForestClassifier(featuresCol='features',labelCol='label', numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=50, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)

results = crossVal(numFold = folds,data = samples, labels = labels,classifier = rf)

avg = {}

for i in results[0].keys():
    avg[i] = 0
    
for i in results.keys():
    for j in results[i].keys():
        avg[j] = avg[j] + results[i][j]
        
for i in avg.keys():
    avg[i] = float(avg[i]/10)

writeResult('allfeatures_rfr',results=avg,category='all') 